In [6]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification

In [7]:
df = pd.read_csv(r'\Users\AIDL-LAB\Desktop\Dashan\dataset\fin.csv', encoding='utf-8')
df

,category,sub_category,crimeaditionalinfo,new_category
0,Financial Fraud Crimes,Fraud CallVishing,The above fraudster is continuously messaging ...,Financial Fraud Crimes
1,Financial Fraud Crimes,Fraud CallVishing,I received a call from lady stating that she w...,Financial Fraud Crimes
2,Financial Fraud Crimes,UPI Related Frauds,FRAUD \t UPI PAYTM \r\nBANK \tPunjab National ...,Financial Fraud Crimes
3,Financial Fraud Crimes,Fraud CallVishing,Sir I am Prabhat Singh jat An app on playstor...,Financial Fraud Crimes
4,Financial Fraud Crimes,Internet Banking Related Fraud,FINANCIAL FRAUD RS,Financial Fraud Crimes
...,...,...,...,...
74099,Financial Fraud Crimes,Debit/Credit Card Fraud,I posted add on olx for labour from west Benga...,Financial Fraud Crimes
74100,Financial Fraud Crimes,Debit/Credit Card Fraud,Victim inform that he purchase some item from ...,Financial Fraud Crimes
74101,Financial Fraud Crimes,Debit/Credit Card Fraud,nameomsinghdistajmerpsramganjpincode\r\ncredit...,Financial Fraud Crimes
74102,Financial Fraud Crimes,Debit/Credit Card Fraud,Sir Mere Cradit Card se bina shoaping kiye hi ...,Financial Fraud Crimes


In [8]:
df['cleaned_text'] = df['crimeaditionalinfo'].str.lower()
df['cleaned_text'] = df['cleaned_text'].str.replace(r'\r\n\t', ' ', regex=True)
df['cleaned_text'] = df['cleaned_text'].str.strip()

In [9]:
df = df.dropna(subset=['crimeaditionalinfo'])
df = df.dropna(subset=['cleaned_text'])

In [10]:
df = df.drop_duplicates()

In [11]:
df['crimeaditionalinfo'].isna().sum()

0

In [12]:
df.groupby('category').size()

category
Financial Fraud Crimes    73285
dtype: int64

In [13]:

df

,category,sub_category,crimeaditionalinfo,new_category,cleaned_text
0,Financial Fraud Crimes,Fraud CallVishing,The above fraudster is continuously messaging ...,Financial Fraud Crimes,the above fraudster is continuously messaging ...
1,Financial Fraud Crimes,Fraud CallVishing,I received a call from lady stating that she w...,Financial Fraud Crimes,i received a call from lady stating that she w...
2,Financial Fraud Crimes,UPI Related Frauds,FRAUD \t UPI PAYTM \r\nBANK \tPunjab National ...,Financial Fraud Crimes,fraud \t upi paytm \r\nbank \tpunjab national ...
3,Financial Fraud Crimes,Fraud CallVishing,Sir I am Prabhat Singh jat An app on playstor...,Financial Fraud Crimes,sir i am prabhat singh jat an app on playstor...
4,Financial Fraud Crimes,Internet Banking Related Fraud,FINANCIAL FRAUD RS,Financial Fraud Crimes,financial fraud rs
...,...,...,...,...,...
74098,Financial Fraud Crimes,Debit/Credit Card Fraud,name ishwar chand phone no gmail id ish...,Financial Fraud Crimes,name ishwar chand phone no gmail id ish...
74099,Financial Fraud Crimes,Debit/Credit Card Fraud,I posted add on olx for labour from west Benga...,Financial Fraud Crimes,i posted add on olx for labour from west benga...
74100,Financial Fraud Crimes,Debit/Credit Card Fraud,Victim inform that he purchase some item from ...,Financial Fraud Crimes,victim inform that he purchase some item from ...
74101,Financial Fraud Crimes,Debit/Credit Card Fraud,nameomsinghdistajmerpsramganjpincode\r\ncredit...,Financial Fraud Crimes,nameomsinghdistajmerpsramganjpincode\r\ncredit...


In [14]:
df.groupby('sub_category').size()

sub_category
Aadhaar Enabled Payment System (AEPS) Fraud     3987
Business Email CompromiseEmail Takeover          380
Debit/Credit Card Fraud                        11956
DematDepository Fraud                            973
EWallet Related Fraud                           5311
Fraud CallVishing                               7589
Internet Banking Related Fraud                 10735
SIM Swap Fraud                                  1300
UPI Related Frauds                             31054
dtype: int64

In [15]:
def resample_data(df, undersample_threshold, oversample_threshold):

    resampled_dfs = []

    counts = df['sub_category'].value_counts()

    for sub_cat, count in counts.items():
        sub_cat_data = df[df['sub_category'] == sub_cat]

        if count > undersample_threshold:
            undersampled_data = sub_cat_data.sample(undersample_threshold, random_state=42)
            resampled_dfs.append(undersampled_data)
        elif count < oversample_threshold:
            oversampled_data = sub_cat_data.sample(oversample_threshold, replace=True, random_state=42)
            resampled_dfs.append(oversampled_data)
        else:
            resampled_dfs.append(sub_cat_data)

    balanced_df = pd.concat(resampled_dfs, ignore_index=True)
    
    return balanced_df

In [16]:
df = resample_data(df,31054,22000) 

In [17]:
df.groupby('sub_category').size()

sub_category
Aadhaar Enabled Payment System (AEPS) Fraud    22000
Business Email CompromiseEmail Takeover        22000
Debit/Credit Card Fraud                        22000
DematDepository Fraud                          22000
EWallet Related Fraud                          22000
Fraud CallVishing                              22000
Internet Banking Related Fraud                 22000
SIM Swap Fraud                                 22000
UPI Related Frauds                             31054
dtype: int64

In [18]:
df.loc[:, 'label'] = df['sub_category'].astype('category').cat.codes
df.groupby('sub_category').describe()

label                           \
                                               count mean  std  min  25%  50%   
sub_category                                                                    
Aadhaar Enabled Payment System (AEPS) Fraud  22000.0  0.0  0.0  0.0  0.0  0.0   
Business Email CompromiseEmail Takeover      22000.0  1.0  0.0  1.0  1.0  1.0   
Debit/Credit Card Fraud                      22000.0  2.0  0.0  2.0  2.0  2.0   
DematDepository Fraud                        22000.0  3.0  0.0  3.0  3.0  3.0   
EWallet Related Fraud                        22000.0  4.0  0.0  4.0  4.0  4.0   
Fraud CallVishing                            22000.0  5.0  0.0  5.0  5.0  5.0   
Internet Banking Related Fraud               22000.0  6.0  0.0  6.0  6.0  6.0   
SIM Swap Fraud                               22000.0  7.0  0.0  7.0  7.0  7.0   
UPI Related Frauds                           31054.0  8.0  0.0  8.0  8.0  8.0   

                                                       
                                             75%  max  
sub_category                                           
Aadhaar Enabled Payment System (AEPS) Fraud  0.0  0.0  
Business Email CompromiseEmail Takeover      1.0  1.0  
Debit/Credit Card Fraud                      2.0  2.0  
DematDepository Fraud                        3.0  3.0  
EWallet Related Fraud                        4.0  4.0  
Fraud CallVishing                            5.0  5.0  
Internet Banking Related Fraud               6.0  6.0  
SIM Swap Fraud                               7.0  7.0  
UPI Related Frauds                           8.0  8.0

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(df['label'].unique()))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
MAX_LEN = 256
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
EPOCHS = 6

In [21]:
texts = df['cleaned_text'].tolist() 
labels = df['label'].tolist()

In [22]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
from torch.optim import AdamW
from torch.optim import AdamW as TorchAdamW 
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch.nn.functional as F


class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.indices = list(range(len(texts))) 

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        index = self.indices[idx]  
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long),
            'indices': torch.tensor(index, dtype=torch.long) 
        }


In [23]:

from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.optim import AdamW as TorchAdamM
from transformers import get_linear_schedule_with_warmup
import torch
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.25, random_state=42
)
train_dataset = TextDataset(train_texts, train_labels, tokenizer, MAX_LEN)
val_dataset = TextDataset(val_texts, val_labels, tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=BATCH_SIZE)
val_loader = DataLoader(val_dataset, sampler=SequentialSampler(val_dataset), batch_size=BATCH_SIZE)

optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=0.01)
total_steps = len(train_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1 * total_steps), num_training_steps=total_steps)

In [24]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
device

device(type='cuda')

In [61]:
from methods import evaluate_model
from functions import train_epoch

best_accuracy = 0

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
 
    train_metrics = train_epoch(model, train_loader, optimizer, scheduler, device)
    train_loss = train_metrics['loss']
    train_accuracy = train_metrics['accuracy']
   
    val_metrics = evaluate_model(model, val_loader, device)
    val_loss = val_metrics['loss']
    val_accuracy = val_metrics['accuracy']
    val_f1 = val_metrics['f1_score']
    val_precision = val_metrics['precision']
    val_recall = val_metrics['recall']

    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1 Score: {val_f1:.4f}, Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}")
    
    if val_accuracy > best_accuracy:
        torch.save(model.state_dict(),'new_____ffc.bin')
        best_accuracy = val_accuracy

print("Training complete.")


Epoch 1/3


Evaluating: 100%|██████████| 2985/2985 [11:45<00:00,  4.23it/s]


Evaluation Metrics:
Loss: 0.6275
Accuracy: 0.7860
Precision: 0.7938
Recall: 0.7860
F1 Score: 0.7822
Train Loss: 1.0561, Train Accuracy: 0.6352
Val Loss: 0.6275, Val Accuracy: 0.7860, Val F1 Score: 0.7822, Val Precision: 0.7938, Val Recall: 0.7860

Epoch 2/3


Evaluating: 100%|██████████| 2985/2985 [11:45<00:00,  4.23it/s]


Evaluation Metrics:
Loss: 0.4138
Accuracy: 0.8692
Precision: 0.8683
Recall: 0.8692
F1 Score: 0.8675
Train Loss: 0.4881, Train Accuracy: 0.8352
Val Loss: 0.4138, Val Accuracy: 0.8692, Val F1 Score: 0.8675, Val Precision: 0.8683, Val Recall: 0.8692

Epoch 3/3


Evaluating: 100%|██████████| 2985/2985 [10:48<00:00,  4.60it/s]


Evaluation Metrics:
Loss: 0.3753
Accuracy: 0.8914
Precision: 0.8897
Recall: 0.8914
F1 Score: 0.8898
Train Loss: 0.2888, Train Accuracy: 0.9074
Val Loss: 0.3753, Val Accuracy: 0.8914, Val F1 Score: 0.8898, Val Precision: 0.8897, Val Recall: 0.8914
Training complete.


In [ ]:
from methods import evaluate_model
from functions import train_epoch

best_accuracy = 0

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
 
    train_metrics = train_epoch(model, train_loader, optimizer, scheduler, device)
    train_loss = train_metrics['loss']
    train_accuracy = train_metrics['accuracy']
   
    val_metrics = evaluate_model(model, val_loader, device)
    val_loss = val_metrics['loss']
    val_accuracy = val_metrics['accuracy']
    val_f1 = val_metrics['f1_score']
    val_precision = val_metrics['precision']
    val_recall = val_metrics['recall']

    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1 Score: {val_f1:.4f}, Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}")
    
    if val_accuracy > best_accuracy:
        torch.save(model.state_dict(),'new_____ffc1.bin')
        best_accuracy = val_accuracy

print("Training complete.")


Epoch 1/6


Training:   0%|          | 0/9706 [00:00<?, ?it/s]

In [41]:
from methods import evaluate_model
from functions import train_epoch

best_accuracy = 0

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
 
    train_metrics = train_epoch(model, train_loader, optimizer, scheduler, device)
    train_loss = train_metrics['loss']
    train_accuracy = train_metrics['accuracy']
   
    val_metrics = evaluate_model(model, val_loader, device)
    val_loss = val_metrics['loss']
    val_accuracy = val_metrics['accuracy']
    val_f1 = val_metrics['f1_score']
    val_precision = val_metrics['precision']
    val_recall = val_metrics['recall']

    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1 Score: {val_f1:.4f}, Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}")
    
    if val_accuracy > best_accuracy:
        torch.save(model.state_dict(),'new_____ffc2.bin')
        best_accuracy = val_accuracy

print("Training complete.")


Epoch 1/3


Evaluating: 100%|██████████| 2735/2735 [09:50<00:00,  4.63it/s]


Evaluation Metrics:
Loss: 0.6499
Accuracy: 0.7776
Precision: 0.7874
Recall: 0.7776
F1 Score: 0.7748
Train Loss: 1.0822, Train Accuracy: 0.6266
Val Loss: 0.6499, Val Accuracy: 0.7776, Val F1 Score: 0.7748, Val Precision: 0.7874, Val Recall: 0.7776

Epoch 2/3


Evaluating: 100%|██████████| 2735/2735 [09:47<00:00,  4.66it/s]


Evaluation Metrics:
Loss: 0.4592
Accuracy: 0.8525
Precision: 0.8505
Recall: 0.8525
F1 Score: 0.8497
Train Loss: 0.5221, Train Accuracy: 0.8234
Val Loss: 0.4592, Val Accuracy: 0.8525, Val F1 Score: 0.8497, Val Precision: 0.8505, Val Recall: 0.8525

Epoch 3/3


Evaluating: 100%|██████████| 2735/2735 [09:46<00:00,  4.66it/s]


Evaluation Metrics:
Loss: 0.4058
Accuracy: 0.8795
Precision: 0.8776
Recall: 0.8795
F1 Score: 0.8779
Train Loss: 0.3203, Train Accuracy: 0.8966
Val Loss: 0.4058, Val Accuracy: 0.8795, Val F1 Score: 0.8779, Val Precision: 0.8776, Val Recall: 0.8795
Training complete.


In [87]:
from methods import evaluate_model
from functions import train_epoch

best_accuracy = 0

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch + 1}/{EPOCHS}")
 
    train_metrics = train_epoch(model, train_loader, optimizer, scheduler, device)
    train_loss = train_metrics['loss']
    train_accuracy = train_metrics['accuracy']
   
    val_metrics = evaluate_model(model, val_loader, device)
    val_loss = val_metrics['loss']
    val_accuracy = val_metrics['accuracy']
    val_f1 = val_metrics['f1_score']
    val_precision = val_metrics['precision']
    val_recall = val_metrics['recall']

    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1 Score: {val_f1:.4f}, Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}")
    
    if val_accuracy > best_accuracy:
        torch.save(model.state_dict(),'new_____ffc3.bin')
        best_accuracy = val_accuracy

print("Training complete.")


Epoch 1/3


Evaluating: 100%|██████████| 3236/3236 [12:24<00:00,  4.35it/s]


Evaluation Metrics:
Loss: 0.3830
Accuracy: 0.8832
Precision: 0.8818
Recall: 0.8832
F1 Score: 0.8821
Train Loss: 0.4088, Train Accuracy: 0.8649
Val Loss: 0.3830, Val Accuracy: 0.8832, Val F1 Score: 0.8821, Val Precision: 0.8818, Val Recall: 0.8832

Epoch 2/3


Evaluating: 100%|██████████| 3236/3236 [11:28<00:00,  4.70it/s]


Evaluation Metrics:
Loss: 0.3448
Accuracy: 0.9090
Precision: 0.9079
Recall: 0.9090
F1 Score: 0.9073
Train Loss: 0.2786, Train Accuracy: 0.9130
Val Loss: 0.3448, Val Accuracy: 0.9090, Val F1 Score: 0.9073, Val Precision: 0.9079, Val Recall: 0.9090

Epoch 3/3


Evaluating: 100%|██████████| 3236/3236 [11:47<00:00,  4.57it/s]


Evaluation Metrics:
Loss: 0.3600
Accuracy: 0.9194
Precision: 0.9184
Recall: 0.9194
F1 Score: 0.9181
Train Loss: 0.1731, Train Accuracy: 0.9501
Val Loss: 0.3600, Val Accuracy: 0.9194, Val F1 Score: 0.9181, Val Precision: 0.9184, Val Recall: 0.9194
Training complete.


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np

def plot_roc_auc(true_labels, all_probs, class_names):
    true_labels_bin = label_binarize(true_labels, classes=range(len(class_names)))
    plt.figure(figsize=(10, 8))

    for i in range(len(class_names)):
        fpr, tpr, _ = roc_curve(true_labels_bin[:, i], all_probs[:, i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], 'k--', label='Random Guess (AUC = 0.5)')

    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()


In [ ]:
class_name = ["Child Pornography CPChild Sexual Abuse Material CSAM",
"RapeGang Rape RGRSexually Abusive Content",
"Sale, Publishing and Transmitting Obscene Material/Sexually Explicit Material",  ]

In [ ]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertForSequenceClassification
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

def evaluate_model_with_roc_and_confusion_matrix(model, data_loader, device, class_names, df):
    model.eval()
    predictions, true_labels = [], []
    total_loss = 0
    all_probs = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="Evaluating"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            indices = batch['indices'].cpu().numpy()  

            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            loss = F.cross_entropy(logits, labels)
            total_loss += loss.item()

            probs = F.softmax(logits, dim=1)
            all_probs.extend(probs.cpu().numpy())
            _, preds = torch.max(logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

            df.loc[indices, 'predictions'] = preds.cpu().numpy()
            df.loc[indices, 'probabilities'] = [list(prob) for prob in probs.cpu().numpy()]

    avg_loss = total_loss / len(data_loader)

    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='weighted')
    recall = recall_score(true_labels, predictions, average='weighted')
    f1 = f1_score(true_labels, predictions, average='weighted')

    print("Evaluation Metrics:")
    print(f"Loss: {avg_loss:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    conf_matrix = confusion_matrix(true_labels, predictions)
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=class_names)
    disp.plot(cmap='Blues', xticks_rotation=45)
    plt.title("Confusion Matrix")
    plt.show()

    all_probs = np.array(all_probs)  
    plot_roc_auc(true_labels, all_probs, class_names)

    return {
        'loss': avg_loss,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'predictions': predictions,
        'true_labels': true_labels,
        'probabilities': all_probs,
        'confusion_matrix': conf_matrix  
    }

In [ ]:
model.load_state_dict(torch.load('best_wcc.bin'))
model = model.to(device)

In [ ]:
evaluate_model_with_roc_and_confusion_matrix(model,val_loader,device,class_name,df)